---
# Imports

In [1]:
# current filepath system leaves plenty to be desired
import sys
local_path = '/Users/hinzlehome/codeup-data-science/spark-exercises/'
sys.path.insert(0, local_path)

# imports.py in /utils/
from utils.imports import *

# plotting magic
%matplotlib inline
# plotting defaults
plt.rc('figure', figsize=(16, 9))
plt.style.use('seaborn-darkgrid')
plt.rc('font', size=16)
# plt.style.available
# ^^^ show available seaborn styles

# !!! Warning !!! 
# *** no more warnings ***
# import warnings
# warnings.filterwarnings("ignore")

# custom mods
# from utils.tidy import *
# from utils.model import *


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


---



In [2]:


np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)



In [3]:
test=pandas_dataframe

In [4]:
test.group.value_counts(normalize=True)

y    0.4
z    0.3
x    0.3
Name: group, dtype: float64

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 11:33:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable



1. Spark Dataframe Basics

    1. Use the starter code above to create a pandas dataframe.
    1. Convert the pandas dataframe to a spark dataframe. From this point
       forward, do all of your work with the spark dataframe, not the pandas
       dataframe.
    1. Show the first 3 rows of the dataframe.
    1. Show the first 7 rows of the dataframe.


In [4]:
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[n: double, group: string, abool: boolean]

In [5]:
spark.range(5).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [6]:
df.show(3)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
+--------------------+-----+-----+
only showing top 3 rows




2. What is the difference between `.show` and `.head`?


In [7]:
df.show

<bound method DataFrame.show of DataFrame[n: double, group: string, abool: boolean]>

In [8]:
df.head()

Row(n=-0.712390662050588, group='z', abool=False)

3. View a summary of the data using `.describe`.


In [9]:
df.describe()

DataFrame[summary: string, n: string, group: string]

4. Use `.select` to create a new dataframe with just the `n` and `abool`
   columns. View the first 5 rows of this dataframe.


In [10]:
df.select('n','abool').show(5)

+--------------------+-----+
|                   n|abool|
+--------------------+-----+
|  -0.712390662050588|false|
|   0.753766378659703|false|
|-0.04450307833805...|false|
| 0.45181233874578974|false|
|  1.3451017084510097|false|
+--------------------+-----+
only showing top 5 rows



5. Use `.select` to create a new dataframe with just the `group` and `abool`
   columns. View the first 5 rows of this dataframe.


In [11]:
df.select('group','abool').show(5)

+-----+-----+
|group|abool|
+-----+-----+
|    z|false|
|    x|false|
|    z|false|
|    y|false|
|    z|false|
+-----+-----+
only showing top 5 rows



6. Use `.select` to create a new dataframe with the `group` column and the
   `abool` column renamed to `a_boolean_value`. Show the first 3 rows of
   this dataframe.


In [12]:
df.select('group',F.col('abool').alias('a_boolean_value')).show(3)

+-----+---------------+
|group|a_boolean_value|
+-----+---------------+
|    z|          false|
|    x|          false|
|    z|          false|
+-----+---------------+
only showing top 3 rows



7. Use `.select` to create a new dataframe with the `group` column and the
   `n` column renamed to `a_numeric_value`. Show the first 6 rows of this
   dataframe.


In [14]:
df.select('group',F.col('n').alias('a_numeric_value')).show(6)

+-----+--------------------+
|group|     a_numeric_value|
+-----+--------------------+
|    z|  -0.712390662050588|
|    x|   0.753766378659703|
|    z|-0.04450307833805...|
|    y| 0.45181233874578974|
|    z|  1.3451017084510097|
|    y|  0.5323378882945463|
+-----+--------------------+
only showing top 6 rows




1. Column Manipulation

1. Use the starter code above to re-create a spark dataframe. Store the
       spark dataframe in a varaible named `df`


In [15]:
df = spark.createDataFrame(pandas_dataframe)


2. Use `.select` to add 4 to the `n` column. Show the results.


In [22]:
df.select('n',df.n+4).show(5)

+--------------------+------------------+
|                   n|           (n + 4)|
+--------------------+------------------+
|  -0.712390662050588|3.2876093379494122|
|   0.753766378659703| 4.753766378659703|
|-0.04450307833805...|3.9554969216619464|
| 0.45181233874578974|  4.45181233874579|
|  1.3451017084510097|5.3451017084510095|
+--------------------+------------------+
only showing top 5 rows




3. Subtract 5 from the `n` column and view the results.

In [23]:
df.select('n',df.n-5).show(5)

+--------------------+-------------------+
|                   n|            (n - 5)|
+--------------------+-------------------+
|  -0.712390662050588| -5.712390662050588|
|   0.753766378659703| -4.246233621340297|
|-0.04450307833805...| -5.044503078338053|
| 0.45181233874578974|  -4.54818766125421|
|  1.3451017084510097|-3.6548982915489905|
+--------------------+-------------------+
only showing top 5 rows



4. Multiply the `n` column by 2. View the results along with the original number

In [24]:
df.select('n',df.n*2).show(5)

+--------------------+--------------------+
|                   n|             (n * 2)|
+--------------------+--------------------+
|  -0.712390662050588|  -1.424781324101176|
|   0.753766378659703|   1.507532757319406|
|-0.04450307833805...|-0.08900615667610691|
| 0.45181233874578974|  0.9036246774915795|
|  1.3451017084510097|  2.6902034169020195|
+--------------------+--------------------+
only showing top 5 rows




5. Add a new column named `n2` that is the `n` value multiplied by -1. Show
       the first 4 rows of your dataframe. You should see the original `n` value
       as well as `n2`.


In [35]:
df.select('n',F.expr('n*-1 AS n2')).show(4)

+--------------------+--------------------+
|                   n|                  n2|
+--------------------+--------------------+
|  -0.712390662050588|   0.712390662050588|
|   0.753766378659703|  -0.753766378659703|
|-0.04450307833805...|0.044503078338053455|
| 0.45181233874578974|-0.45181233874578974|
+--------------------+--------------------+
only showing top 4 rows



6. Add a new column named `n3` that is the n value squared. Show the first 5
       rows of your dataframe. You should see both `n`, `n2`, and `n3`.


In [36]:
df.select('n',F.expr('n*-1 AS n2'),F.expr('pow(n,2) AS n3')).show(4)

+--------------------+--------------------+--------------------+
|                   n|                  n2|                  n3|
+--------------------+--------------------+--------------------+
|  -0.712390662050588|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|-0.45181233874578974| 0.20413438944294027|
+--------------------+--------------------+--------------------+
only showing top 4 rows




    7. What happens when you run the code below?


In [37]:
df.group + df.abool



Column<'(group + abool)'>


    8. What happens when you run the code below? What is the difference between
       this and the previous code sample?


In [38]:

# df.select(df.group + df.abool)

# this code tries to execute an action, so theres an error cuz type mismatch

AnalysisException: cannot resolve '(CAST(group AS DOUBLE) + abool)' due to data type mismatch: differing types in '(CAST(group AS DOUBLE) + abool)' (double and boolean).;
'Project [unresolvedalias((cast(group#199 as double) + abool#200), Some(org.apache.spark.sql.Column$$Lambda$3950/0x0000000801616040@28ce92f))]
+- LogicalRDD [n#198, group#199, abool#200], false



    9. Try adding various other columns together. What are the results of
       combining the different data types?


In [39]:
# df.select(df.n + df.abool)

AnalysisException: cannot resolve '(n + abool)' due to data type mismatch: differing types in '(n + abool)' (double and boolean).;
'Project [unresolvedalias((n#198 + abool#200), Some(org.apache.spark.sql.Column$$Lambda$3950/0x0000000801616040@28ce92f))]
+- LogicalRDD [n#198, group#199, abool#200], false


In [43]:
df.select(df.n + df.group).show(5)

+-----------+
|(n + group)|
+-----------+
|       null|
|       null|
|       null|
|       null|
|       null|
+-----------+
only showing top 5 rows




3. Type casting

    1. Use the starter code above to re-create a spark dataframe.


In [44]:
df = spark.createDataFrame(pandas_dataframe)


    2. Use `.printSchema` to view the datatypes in your dataframe.


In [48]:
df.printSchema()

root
 |-- n: double (nullable = true)
 |-- group: string (nullable = true)
 |-- abool: boolean (nullable = true)




    3. Use `.dtypes` to view the datatypes in your dataframe.


In [52]:
df.dtypes

[('n', 'double'), ('group', 'string'), ('abool', 'boolean')]


    4. What is the difference between the two code samples below?


In [56]:

df.abool.cast('int')

# this returns a column object

Column<'CAST(abool AS INT)'>

In [58]:
df.select(df.abool.cast('int')).show(5)

# this returns a column print

+-----+
|abool|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows




    5. Use `.select` and `.cast` to convert the `abool` column to an integer
       type. View the results.


In [59]:
df.select(df.abool.cast('int')).show(5)

+-----+
|abool|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows



    6. Convert the `group` column to a integer data type and view the results.
       What happens?


In [61]:
df.select(df.group.cast('int')).show(5)

# all nulls

+-----+
|group|
+-----+
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 5 rows



    7. Convert the `n` column to a integer data type and view the results. What
       happens?


In [64]:
df.select(df.n.cast('int')).show(5)

# all ints

+---+
|  n|
+---+
|  0|
|  0|
|  0|
|  0|
|  1|
+---+
only showing top 5 rows



    8. Convert the `abool` column to a string data type and view the results.
       What happens?


In [66]:
df.select(df.abool.cast('string')).show(5)

# all strings


+-----+
|abool|
+-----+
|false|
|false|
|false|
|false|
|false|
+-----+
only showing top 5 rows




4. Built-in Functions

    1. Use the starter code above to re-create a spark dataframe.


In [68]:
df = spark.createDataFrame(pandas_dataframe)

    2. Import the necessary functions from `pyspark.sql.functions`

done from imports

imported as 'F'

    3. Find the highest `n` value.


In [78]:
df.select(F.max('n')).show()

+------------------+
|            max(n)|
+------------------+
|2.1503829673811126|
+------------------+



    4. Find the lowest `n` value.


In [80]:
df.select(F.min('n')).show()

+------------------+
|            min(n)|
+------------------+
|-1.261605945319069|
+------------------+



    5. Find the average `n` value.


In [81]:
df.select(F.avg('n')).show()

+------------------+
|            avg(n)|
+------------------+
|0.3664026449885216|
+------------------+



    6. Use `concat` to change the `group` column to say, e.g. "Group: x" or
       "Group: y"


In [98]:
df.select(F.concat(F.lit('Group: '),df.group).alias('Group')).show(5)

+--------+
|   Group|
+--------+
|Group: z|
|Group: x|
|Group: z|
|Group: y|
|Group: z|
+--------+
only showing top 5 rows



    7. Use `concat` to combine the `n` and `group` columns to produce results
       that look like this: "x: -1.432" or "z: 2.352"


In [102]:
df.select(F.concat(df.group,F.lit(': '),df.n).alias('Group')).show(5,truncate=False)

+------------------------+
|Group                   |
+------------------------+
|z: -0.712390662050588   |
|x: 0.753766378659703    |
|z: -0.044503078338053455|
|y: 0.45181233874578974  |
|z: 1.3451017084510097   |
+------------------------+
only showing top 5 rows




5. When / Otherwise

    1. Use the starter code above to re-create a spark dataframe.


In [103]:
df = spark.createDataFrame(pandas_dataframe)

    2. Use `when` and `.otherwise` to create a column that contains the text "It
       is true" when `abool` is true and "It is false" when `abool` is false.


In [113]:
df.select('abool',F.when(df.abool == True, "It is true").otherwise("It is false").alias("It is:")).show(10)

+-----+-----------+
|abool|     It is:|
+-----+-----------+
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
| true| It is true|
| true| It is true|
+-----+-----------+
only showing top 10 rows



    3. Create a column that contains 0 if n is less than 0, otherwise, the
       original n value.


In [122]:
df.select('n', F.when(df.n<0,0).otherwise(df.n).alias("short title")).show(5)

+--------------------+-------------------+
|                   n|        short title|
+--------------------+-------------------+
|  -0.712390662050588|                0.0|
|   0.753766378659703|  0.753766378659703|
|-0.04450307833805...|                0.0|
| 0.45181233874578974|0.45181233874578974|
|  1.3451017084510097| 1.3451017084510097|
+--------------------+-------------------+
only showing top 5 rows




1. Filter / Where

    1. Use the starter code above to re-create a spark dataframe.


In [123]:
df = spark.createDataFrame(pandas_dataframe)

    2. Use `.filter` or `.where` to select just the rows where the group is `y`
       and view the results.


In [127]:
df.filter(df.group=='y').show(5)

+-------------------+-----+-----+
|                  n|group|abool|
+-------------------+-----+-----+
|0.45181233874578974|    y|false|
| 0.5323378882945463|    y|false|
|-1.0453771305385342|    y| true|
| -1.261605945319069|    y|false|
| 0.5628467852810314|    y| true|
+-------------------+-----+-----+
only showing top 5 rows



    3. Select just the columns where the `abool` column is false and view the
       results.


In [128]:
df.filter(df.abool==False).show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
+--------------------+-----+-----+
only showing top 5 rows



    4. Find the columns where the `group` column is *not* `y`.


In [130]:
df.where(df.group!='y').show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
+--------------------+-----+-----+
only showing top 5 rows



    5. Find the columns where `n` is positive.


In [131]:
df.filter(df.n>0).show(5)

+-------------------+-----+-----+
|                  n|group|abool|
+-------------------+-----+-----+
|  0.753766378659703|    x|false|
|0.45181233874578974|    y|false|
| 1.3451017084510097|    z|false|
| 0.5323378882945463|    y|false|
| 1.3501878997225267|    z|false|
+-------------------+-----+-----+
only showing top 5 rows



    6. Find the columns where `abool` is true and the `group` column is `z`.


In [134]:
df.filter(df.n>0).filter(df.group=='z').show(5)

+-------------------+-----+-----+
|                  n|group|abool|
+-------------------+-----+-----+
| 1.3451017084510097|    z|false|
| 1.3501878997225267|    z|false|
| 1.4786857374358966|    z| true|
|0.12730328020698067|    z|false|
+-------------------+-----+-----+



    7. Find the columns where `abool` is true or the `group` column is `z`.


In [135]:
df.where(df.abool==True).filter(df.group=='z').show(5)

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|1.4786857374358966|    z| true|
+------------------+-----+-----+



    8. Find the columns where `abool` is false and `n` is less than 1


In [136]:
df.where(df.abool==False).where(df.n<1).show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
+--------------------+-----+-----+
only showing top 5 rows



    9. Find the columns where `abool` is false or `n` is less than 1


In [140]:
df.filter((df.abool==False)|(df.n<1)).show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
+--------------------+-----+-----+
only showing top 5 rows




7. Sorting

    1. Use the starter code above to re-create a spark dataframe.


In [141]:
df = spark.createDataFrame(pandas_dataframe)

    2. Sort by the `n` value.


In [145]:
df.sort('n').show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
| -0.7889890249515489|    x|false|
|  -0.712390662050588|    z|false|
|-0.24332625188556253|    y| true|
+--------------------+-----+-----+
only showing top 5 rows



    3. Sort by the `group` value, both ascending and descending.


In [153]:
df.sort(df['group'].desc()).show(5),df.sort(df.group.asc()).show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|  1.4786857374358966|    z| true|
|-0.04450307833805...|    z|false|
| 0.12730328020698067|    z|false|
|  1.3501878997225267|    z|false|
+--------------------+-----+-----+
only showing top 5 rows

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|   0.753766378659703|    x|false|
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
| -0.7889890249515489|    x|false|
| 0.31735092273633597|    x|false|
+--------------------+-----+-----+
only showing top 5 rows



(None, None)

    4. Sort by the group value first, then, within each group, sort by `n`
       value.


In [163]:
df.sort('group','n').show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
| -0.7889890249515489|    x|false|
|-0.02677164998644...|    x| true|
| 0.31735092273633597|    x|false|
|  0.6062886568962988|    x|false|
|   0.753766378659703|    x|false|
|  0.8612113741693206|    x|false|
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
|-0.24332625188556253|    y| true|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.5628467852810314|    y| true|
|  0.9137407048596775|    y|false|
|  2.1503829673811126|    y| true|
|  -0.712390662050588|    z|false|
|-0.04450307833805...|    z|false|
| 0.12730328020698067|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
+--------------------+-----+-----+



    5. Sort by `abool`, `group`, and `n`. Does it matter in what order you
       specify the columns when sorting?


In [167]:
df.sort('abool','group','n').show(5)

# order matters

+-------------------+-----+-----+
|                  n|group|abool|
+-------------------+-----+-----+
|-0.7889890249515489|    x|false|
|0.31735092273633597|    x|false|
| 0.6062886568962988|    x|false|
|  0.753766378659703|    x|false|
| 0.8612113741693206|    x|false|
+-------------------+-----+-----+
only showing top 5 rows




1. Aggregating

    1. What is the average `n` value for each group in the `group` column?
    

2. What is the maximum `n` value for each group in the `group` column?
    3. What is the minimum `n` value by `abool`?
    4. What is the average `n` value for each unique combination of the `group`
       and `abool` column?

1.  Spark SQL

    1. Use the starter code above to re-create a spark dataframe.
    2. Turn your dataframe into a table that can be queried with spark SQL. Name
       the table `my_df`. Answer the rest of the questions in this section with
       a spark sql query (`spark.sql`) against `my_df`. After each step, view
       the first 7 records from the dataframe.
    3. What happens if you make a SQL syntax error in your query?
    4. Write a query that shows all of the columns from your dataframe.
    5. Write a query that shows just the `n` and `abool` columns from the
       dataframe.
    6. Write a query that shows just the `n` and `group` columns. Rename the
       `group` column to `g`.
    7. Write a query that selects `n`, and creates two new columns: `n2`, the
       original `n` values halved, and `n3`: the original n values minus 1.